# Quick Start

## Choosing between calibration models and calculating concentrations

In [11]:
from calipytion.tools import Calibrator
from calipytion.units import mM, C

In [2]:
# Some measured data of an ABTS standard
concentrations = [0.0, 5.0, 10.0, 15.0, 25.0, 50.0, 75.0, 100.0, 150.0, 200.0]
absorption = [
    0.0011,
    0.0790,
    0.1652,
    0.2502,
    0.4306,
    0.9713,
    1.4482,
    1.9507,
    2.8796,
    3.5045,
]

# Create a calibrator object
calibrator = Calibrator(
    molecule_id="https://pubchem.ncbi.nlm.nih.gov/compound/5464076",
    molecule_name="ABTS",
    molecule_symbol="s1",
    concentrations=concentrations,
    signals=absorption,
    conc_unit=mM,
)

In [3]:
# add equation for limited growth
# lim_model = calibrator.add_model(
#     name="limited growth",
#     signal_law="A_max * (1 - exp(-k * s1))",
# )

In [4]:
calibrator.fit_models()

✅ Models have been fitted successfully.


                                                Model Overview                                                 
┏━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Model Name ┃ AIC ┃ R squared ┃ RMSD   ┃ Equation                       ┃ Relative Parameter Standard Errors ┃
┡━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ quadratic  │ -58 │ 0.9988    │ 0.0417 │ a * s1 + b * s1**2 + c         │ a: 0.1%, b: 0.0%, c: 2.7%,         │
│ linear     │ -48 │ 0.9953    │ 0.0817 │ a * s1                         │ a: 0.0%,                           │
│ cubic      │ 15  │ -0.6877   │ 1.5403 │ a * s1 + b * s1**2 + c * s1**3 │ a: n.a., b: n.a., c: n.a.,         │
└────────────┴─────┴───────────┴────────┴────────────────────────────────┴────────────────────────────────────┘

In [7]:
calibrator.visualize()

![svg](ABTS_calibration_curve.svg)

## Calculate concentrations of unknowns

In [8]:
unknowns = [0.12, 1.4, 2, 3, 3.9]

linear_model = calibrator.get_model("linear")
calibrator.calculate_concentrations(linear_model, unknowns, extrapolate=False)

⚠️ Some concentrations could not be calculated and were replaced with nan values, since the provided signal is outside the calibration range. To calculate the concentration outside the calibration range, set 'extrapolate=True'.


[6.520805389870049,
 76.07606288181724,
 108.68008983116748,
 163.02013474675124,
 nan]

In [13]:
standard_report = calibrator.create_standard(
    linear_model, ph=3, temperature=25, temp_unit=C
)

# save to file
with open("abts_calibration_report.json", "w") as f:
    f.write(standard_report.model_dump_json(indent=4))

## Convert EnzymeML